# Suspicious Encoded PowerShell Command Line
Detects suspicious powershell process starts with base64 encoded commands (e.g. Emotet)

## Rule Content
```
- title: Suspicious Encoded PowerShell Command Line
  id: ca2092a1-c273-4878-9b4b-0d60115bf5ea
  description: Detects suspicious powershell process starts with base64 encoded commands
    (e.g. Emotet)
  status: experimental
  references:
  - https://app.any.run/tasks/6217d77d-3189-4db2-a957-8ab239f3e01e
  author: Florian Roth, Markus Neis
  date: 2018/09/03
  modified: 2019/12/16
  tags:
  - attack.execution
  - attack.t1086
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - '* -e JAB*'
      - '* -e  JAB*'
      - '* -e   JAB*'
      - '* -e    JAB*'
      - '* -e     JAB*'
      - '* -e      JAB*'
      - '* -en JAB*'
      - '* -enc JAB*'
      - '* -enc* JAB*'
      - '* -w hidden -e* JAB*'
      - '* BA^J e-'
      - '* -e SUVYI*'
      - '* -e aWV4I*'
      - '* -e SQBFAFgA*'
      - '* -e aQBlAHgA*'
      - '* -enc SUVYI*'
      - '* -enc aWV4I*'
      - '* -enc SQBFAFgA*'
      - '* -enc aQBlAHgA*'
    falsepositive1:
      CommandLine: '* -ExecutionPolicy remotesigned *'
    condition: selection and not falsepositive1
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:(*\ \-e\ JAB* OR *\ \-e\ \ JAB* OR *\ \-e\ \ \ JAB* OR *\ \-e\ \ \ \ JAB* OR *\ \-e\ \ \ \ \ JAB* OR *\ \-e\ \ \ \ \ \ JAB* OR *\ \-en\ JAB* OR *\ \-enc\ JAB* OR *\ \-enc*\ JAB* OR *\ \-w\ hidden\ \-e*\ JAB* OR *\ BA\^J\ e\- OR *\ \-e\ SUVYI* OR *\ \-e\ aWV4I* OR *\ \-e\ SQBFAFgA* OR *\ \-e\ aQBlAHgA* OR *\ \-enc\ SUVYI* OR *\ \-enc\ aWV4I* OR *\ \-enc\ SQBFAFgA* OR *\ \-enc\ aQBlAHgA*) AND (NOT (process_command_line.keyword:*\ \-ExecutionPolicy\ remotesigned\ *)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()